Downoald data set from competition

<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=down>
</div>


KUL H02A5a Computer Vision: Group Assignment 1
---------------------------------------------------------------
Student numbers: <span style="color:red">r0893875, r0877856, r0745139, r0880442</span>.

The goal of this assignment is to explore more advanced techniques for constructing features that better describe objects of interest and to perform face recognition using these features. This assignment will be delivered in groups of 5 (either composed by you or randomly assigned by your TA's).

In this assignment you are a group of computer vision experts that have been invited to ECCV 2021 to do a tutorial about  "Feature representations, then and now". To prepare the tutorial you are asked to participate in a kaggle competition and to release a notebook that can be easily studied by the tutorial participants. Your target audience is: (master) students who want to get a first hands-on introduction to the techniques that you apply.

---------------------------------------------------------------
This notebook is structured as follows:
0. Data loading & Preprocessing
1. Feature Representations
2. Evaluation Metrics 
3. Classifiers
4. Experiments
5. Publishing best results
6. Discussion

Make sure that your notebook is **self-contained** and **fully documented**. Walk us through all steps of your code. Treat your notebook as a tutorial for students who need to get a first hands-on introduction to the techniques that you apply. Provide strong arguments for the design choices that you made and what insights you got from your experiments. Make use of the *Group assignment* forum/discussion board on Toledo if you have any questions.

Fill in your student numbers above and get to it! Good luck! 

# 0. Data loading & Preprocessing

We load and preprocess the VGG faces dataset. The training data set contains:

- 30 images of Mila Kunis
- 30 images of Jesse Eisenberg
- 10 images of Michael Cera (who looks like Jesse Eisenberg)
- 10 images of Sarah Hyland (who looks like Mila Kunis).

The test set contains 1816 examples. 

Our goal is to build a **classifier** for face recognition of Mila and Jesse. 
## 0.0. Loading libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dlib
from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library

## 0.1. Loading data
The training set is many times smaller than the test set and this might strike you as odd, however, this is close to a **real world** scenario where your system might be put through daily use! In this session we will try to do the best we can with the data that we've got! 

In [2]:
# Input data files are available in the read-only "../input/" directory

train = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2022/train_set.csv', index_col = 0)
train.index = train.index.rename('id')

test = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2022/test_set.csv', index_col = 0)
test.index = test.index.rename('id')

# read the images as numpy arrays and store in "img" column
train['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2022/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in train.iterrows()]

test['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2022/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in test.iterrows()]
  

train_size, test_size = len(train),len(test)

"The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

# Input data files are available in the read-only "../input/" directory

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

The train set is a panda.Dataframe where each sample has four attributes: 'id', 'name', 'class', 'img'. The classes are:

- Jesse: 1
- Mila: 2
- Michael or Sarah: 0

The images are all RBG and have random sizes. At this point, we already understand that resizing all the images and working in monochrome will be helpful.  

<img src="https://drive.google.com/uc?id=1fhZ61sk96xQS82BxoQhFNSsijGIa5CGx " width="200"/> <img src="https://drive.google.com/uc?id=1AtkvZr1BfqzehjSkgZUcITW5-j_g3FW_" width="200"/> <img src="https://drive.google.com/uc?id=1_a6D6nyWgkLwZ4tHd4HgPI0UYb-1LeVD" width="200"/> <img src="https://drive.google.com/uc?id=1h-1YsdFlDVzj65aWTTjoREj0ZQxhCQDZ" width="200"/> 


In [3]:
im = train['img'][0]
imRGB = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
cv2.imwrite('./im2.png', imRGB)


In [4]:
train.keys

In [ ]:
print(train['name'].to_string())

## 0.3. Preprocess data
### 0.3.1 Example: dlib face detector


<!-- <div class="alert alert-block alert-info"> <b>NOTE:</b> You can write temporary files to <code>/kaggle/temp/</code> or <code>../../tmp</code>, but they won't be saved outside of the current session
</div> -->


An important optimization in the pipeline is preprocessing. It makes sure that our raw sample images are formatted to agree on a number of properties;

1. Fixed resolution
2. Normalized lighting
3. Aligned eyes (by rotation)
4. Fixed eye positions (by translation and scaling)

To achieve these properties, the preprocessing performs the following steps:

1. Convert the image to grayscale.
2. Try to detect faces in the image. If it does not contain exactly one face, assign '0'.
3. Translation & rotation
4. Light normalization & face cropping

This process is shown by two example pictures below. On the left you see the original raw image. On the right you the output of the preprocessing. After the preprocessing we filter out the 'imposter' images from the training batch, that is all images which got assigned '0'.

The face-and eye detection package that was used can be found here:
https://towardsdatascience.com/facial-mapping-landmarks-with-dlib-python-160abcf7d672

In [3]:
from urllib.request import urlretrieve
import bz2, shutil

# Load file used to detect facial landmarks
url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
filename = 'shape_predictor_68_face_landmarks.dat.bz2'

def download(url, file):
    print(os.path.isfile(file))
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

file = download(url,filename)


with bz2.BZ2File(filename) as input:
    with open('shape_predictor_68_face_landmarks.dat', 'wb') as output:
        shutil.copyfileobj(input, output)
  

In [5]:
      
url_2 = 'http://dlib.net/files/mmod_human_face_detector.dat.bz2'
filename_2 = 'mmod_human_face_detector.dat.bz2'
file_2 = download(url_2,filename_2)

with bz2.BZ2File(filename_2) as input:
    with open('mmod_human_face_detector.dat', 'wb') as output:
        shutil.copyfileobj(input, output)

Let us choose if we will work with Gray or RGB images

In [8]:
gray_var = True
channel = None # if channel = -1 --> RGB images, if channel = None --> gray images

In [13]:
import matplotlib.patches as patches
from scipy import ndimage
import math

l_eye_loc = 0.3

# detector = dlib.get_frontal_face_detector()
detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def preprocess(img, size, Gray, plot):
    norm = 100
    gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2GRAY)
    
    if Gray == False:
        gray = img
        norm = 255

    faces = [convert_and_trim_bb(image, r) for r in faces]
    if len(faces) ==0:
        return cv2.resize(gray,(size,size),interpolation=cv2.INTER_LINEAR ), 0
    
    if len(faces) >1:
        return cv2.resize(gray,(size,size),interpolation=cv2.INTER_LINEAR), 2
    
    (x, y, w, h) = faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()
    shape=predictor(gray,faces[0])
    
    #eyes 
    #right eye, list of all landmarks, then compute center of mass
    re_xvalues = []
    re_yvalues=[]
    for i in range(36,42):
        re_xvalues.append(shape.part(i).x)
        re_yvalues.append(shape.part(i).y)
        
    le_xvalues = []
    le_yvalues=[]
    for i in range(42,48):
        le_xvalues.append(shape.part(i).x)
        le_yvalues.append(shape.part(i).y)     
        
    #center of mass
    left_eye = [np.mean(le_xvalues), np.mean(le_yvalues)]
    right_eye = [np.mean(re_xvalues), np.mean(re_yvalues)]
    
    dX=left_eye[0]-right_eye[0]
    dY = left_eye[1]-right_eye[1]
    angle = np.arctan(dY/dX)
        
    #calculate translation
    r_eye_loc = 1-l_eye_loc
    
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = r_eye_loc-l_eye_loc
    desiredDist *= 256
    scale = desiredDist / dist
    
    eye_center = ((left_eye[0]+right_eye[0])//2 , (left_eye[1]+right_eye[1])//2)
    
    M = cv2.getRotationMatrix2D(eye_center, math.degrees(angle), scale)
    
    tX = 256 * 0.5
    tY = 256 * l_eye_loc
    M[0, 2] += (tX - eye_center[0])
    M[1, 2] += (tY - eye_center[1])
    
    img_rotated = cv2.warpAffine(gray, M, (256, 256)) 

    
    #normalize
    norm_img = np.zeros((size, size))
    norm_img = cv2.normalize(img_rotated, norm_img, 0, norm, cv2.NORM_MINMAX)
    
    cropped_im= cv2.resize(norm_img,(size,size),interpolation=cv2.INTER_LINEAR )
    
    if plot:
        fig, (ax1, ax2 ) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(img)
        ax1.set_title('Raw Image', fontsize=18)
        ax2.imshow(cropped_im, cmap='gray')
        ax2.set_title('Preprocessed Image', fontsize=30)
    return cropped_im, 1

In [15]:
preprocess(train['img'][4], 100, gray_var, True)

## Preprocessing

To make full use of the given training set we continue for pictures on which there are more than one face recognized. Below is function that will add faces to the training set, we manually labelled these extra faces.

In [277]:
def preprocess_m(img, size,Gray):
    norm = 100
    gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2GRAY)
    if Gray == False:
        gray = img
        norm = 255
        
    #face
    face_list = []
    faces = detector(gray, 1)
    for face in faces:
    
        (x, y, w, h) = face.left(), face.top(), face.width(), face.height()
        shape=predictor(gray,face)
        #eyes 
        #right eye, list of all landmarks, then compute center of mass
        re_xvalues = []
        re_yvalues=[]
        for i in range(36,42):
            re_xvalues.append(shape.part(i).x)
            re_yvalues.append(shape.part(i).y)
            
        le_xvalues = []
        le_yvalues=[]
        for i in range(42,48):
            le_xvalues.append(shape.part(i).x)
            le_yvalues.append(shape.part(i).y)     
            
        #center of mass
        left_eye = [np.mean(le_xvalues), np.mean(le_yvalues)]
        right_eye = [np.mean(re_xvalues), np.mean(re_yvalues)]
        
        dX=left_eye[0]-right_eye[0]
        dY = left_eye[1]-right_eye[1]
        angle = np.arctan(dY/dX)
        
        #calculate translation
        r_eye_loc = 1-l_eye_loc
        
        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = r_eye_loc-l_eye_loc
        desiredDist *= 256
        scale = desiredDist / dist
        
        eye_center = ((left_eye[0]+right_eye[0])//2 , (left_eye[1]+right_eye[1])//2)
        
        M = cv2.getRotationMatrix2D(eye_center, math.degrees(angle), scale)
        
        tX = 256 * 0.5
        tY = 256 * l_eye_loc
        M[0, 2] += (tX - eye_center[0])
        M[1, 2] += (tY - eye_center[1])
        
        img_rotated = cv2.warpAffine(gray, M, (256, 256)) 
        
        #normalize
        norm_img = np.zeros((size, size))
        norm_img = cv2.normalize(img_rotated, norm_img, 0, norm, cv2.NORM_MINMAX)
        
        cropped_im= cv2.resize(norm_img,(size,size),interpolation=cv2.INTER_LINEAR )
        face_list.append(cropped_im)
    return face_list

Some plotting functions

In [10]:
def plot_image_sequence(data, imgs_per_column, imgs_per_row):
    fig = plt.figure(figsize=(100, 100))
    columns = imgs_per_row
    rows = imgs_per_column
    for i in range(len(data)):
        img = data[i]
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(img)
    plt.show()

In [11]:
def plot_image_sequence_advanced(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i],cmap='gray')
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i],cmap='gray')
        else:
            ax[int(i%n)].imshow(data[i],cmap='gray')
    plt.show()

In [296]:
# Preprocessed data 

number_of_test_samples = 100
size=100
train_X_pr = [preprocess(img,size, gray_var, False) for img in train['img']]
test_X_pr = [preprocess(img,size, gray_var, False) for img in test['img'][:number_of_test_samples]]

# Collect all train class label
train_y = np.array([train['class'].values[i] for i in range(len(train['class'].values))
           if train_X_pr[i][1]==1])

# Based on labeling only go further with pictures with only one face on it
train_X = np.array([img[0] for img in train_X_pr if img[1]==1]) # If img[1] == 1 --> one face, img[1] == 2 --> more than 1 face, img[1] == 0 --> 0 faces

# Collect all test images
test_X = [img[0] for img in test_X_pr]

Handcrafted test label list of first 100 samples to test accuracy at the end

In [13]:
test_y_acc =[1,0,0,0,1,1,1,0,1,0,2,1,2,0,0,1,1,0,0,1,2,2,2,1,0,1,0,0,0,2,2,1,0,0,2,1,1,0,0,1,0,1,1,1,0,1,1,0,0,0,1,0,0,0,2,0,2,0,0,1,2,0,0,2,0,0,0,0,1,0,0,2,0,0,1,1,0,2,1,0,0,0,2,0,0,2,0,1,2,2,0,0,0,0,2,0,0,0,1,2]

Replace class 0 by two new classes: 3 = Michael (--> g_ind), 4 = Sarah (--> f_ind)

The goal of creating this new classes is to increase the accuracy

In [297]:
# Two different sets of list depending if we are using color or grayscale images (because with color images, the algorithm detect one more multiple faces picture)
if gray_var:
    g_ind = [2, 4, 15, 16, 24, 39, 58, 63, 65]
    f_ind = [11,20,32, 37, 42, 46,49,52]
else:
    g_ind = [2, 4, 15, 16, 24, 38, 57, 62, 64]
    f_ind = [11,20,31, 36, 41, 45,48,51]

# Let us updates the label with the new classes
updated_labels=train_y

for i in g_ind:
    updated_labels[i]=3
for i in f_ind:
    updated_labels[i]=4

We now create new datasets with the new classes and we add the new figures found on multiple faces picture to increase the train dataset

In [307]:
# Add data of multiple faces images to training set (we have thanks to this, 19 new pictures added to the train dataset)


if not gray_var:
    train_X_full = np.zeros((len(updated_labels) + 19, size, size,3)).astype(int)
    train_y_full = np.zeros(len(updated_labels) + 19)
    train_y_full[:len(updated_labels)] = updated_labels
else:
    train_X_full = np.zeros((len(updated_labels) + 17, size, size)).astype(int)
    train_X_full[:len(updated_labels)] = train_X
    train_y_full = np.zeros(len(updated_labels) + 17)
    train_y_full[:len(updated_labels)] = updated_labels

Let us now add these new pictures to the training dataset. We checked each original multiple faces picture index to extract them more easily

In [308]:
# We add those new faces to the train dataset with their corresponding label

for i, el in enumerate(train_X_pr):
    if i==18:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)] = r[0]
        train_y_full[len(train_X)] = 3
        
        train_X_full[len(train_X)+1] = r[1]
        train_y_full[len(train_X)+1] = 1
    elif i==28:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+2] = r[0]
        train_y_full[len(train_X)+2] = 2
    elif i==30:
        # Multiple faces picture not detected when we work with grayscale images
        if not gray_var:
            r = preprocess_m(train['img'][i], size, Gray=gray_var)
            train_X_full[len(train_X)+17] = r[0]
            train_y_full[len(train_X)+17] = 2

            train_X_full[len(train_X)+18] = r[1]
            train_y_full[len(train_X)+18] = 2
    elif i==32:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+3] = r[0]
        train_y_full[len(train_X)+3] = 4
        
        train_X_full[len(train_X)+4] = r[1]
        train_y_full[len(train_X)+4] = 4
    elif i==34:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+5] = r[1]
        train_y_full[len(train_X)+5] = 3
        
        train_X_full[len(train_X)+6] = r[2]
        train_y_full[len(train_X)+6] = 1
        
    elif i==41:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+7] = r[1]
        train_y_full[len(train_X)+7] = 3
        
    elif i==49:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+8] = r[1]
        train_y_full[len(train_X)+8] = 1
        
    elif i==50:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+9] = r[0]
        train_y_full[len(train_X)+9] = 2
        
    elif i==52:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+10] = r[1]
        train_y_full[len(train_X)+10] = 1
        
    elif i==53:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+11] = r[0]
        train_y_full[len(train_X)+11] = 1
        
    elif i==59:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+12] = r[0]
        train_y_full[len(train_X)+12] = 4
        
    elif i==61:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+13] = r[0]
        train_y_full[len(train_X)+13] = 2
        
    elif i==70:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+14] = r[1]
        train_y_full[len(train_X)+14] = 1
        
    elif i==77:
        r = preprocess_m(train['img'][i], size, Gray=gray_var)
        train_X_full[len(train_X)+15] = r[0]
        train_y_full[len(train_X)+15] = 2
        
        train_X_full[len(train_X)+16] = r[1]
        train_y_full[len(train_X)+16] = 2

Save indexes of no faces, one face and multiple faces pictures for the test dataset (it will be used latter to calculate the predictions)

In [242]:

# Screening of test data

# Zero faced images
bool_list = np.array([img[1] for img in test_X_pr])==0
zeros_ind = [i for i, x in enumerate(bool_list) if x]

# One-faced images
bool_list = np.array([img[1] for img in test_X_pr])==1
face_ind = [i for i, x in enumerate(bool_list) if x]

# More than one-faced images
bool_list = np.array([img[1] for img in test_X_pr])==2
faces_ind = [i for i, x in enumerate(bool_list) if x]


# Final classes prediction of test dataset
labels = np.zeros(len(test_X_pr))

Let us increase our initial dataset with data_augmentation to improve our predictions

In [18]:
# import library for data augmentation
import tensorflow as tf

In [109]:
def data_augmentation_func(images, classes):
    """
    Increase the initial train dataset

    Input: initial train dataset
    Output: augmented train dataset
    """
    
    augmented_images = images.tolist()
    augmented_classes = classes.tolist()

    for i, image in enumerate(images):
        if not gray_var:
            # Let us add grayscale images of the initial train dataset
            image_ = tf.image.rgb_to_grayscale(image)
            image_ = tf.squeeze(image_)
            image_ = cv2.cvtColor(np.uint8(image_),cv2.COLOR_GRAY2RGB)
        else:
            # If we are working with grayscale images let use add some saturated images
            image_ = tf.image.adjust_saturation(image, 3)
        augmented_images.append(image_)
        augmented_classes.append(classes[i])
        
        # Let us add pictures with brightness changes of the initial train dataset
        bright = tf.image.adjust_brightness(image, 0.4)
        augmented_images.append(bright)
        augmented_classes.append(classes[i])
        
        # Let us add some saturated images
        saturated = tf.image.adjust_saturation(image, 3)
        augmented_images.append(saturated)
        augmented_classes.append(classes[i])
    return [np.asarray(augmented_images), augmented_classes]

We notice that we had better performances by adding grayscale images and brighter images to the initial train dataset

Let us now create our new augmented train dataset

In [125]:
output = data_augmentation_func(train_X_full,train_y_full)
train_X_full_trans = output[0]
train_y_full_trans = output[1]

# 1. Feature Representations

In this section, we will explore two methods to extract features from images. As we're concerned with processing faces, these features include various facial properties such as eyes, mouths, noses, ears, etc. We will look at both handcrafted features and learned features.

Handcrafted feature descriptors are extracted from the image itself, using a specific deterministic algorithm that might differ from task to task. In this assignment we will look at Histogram of Gradients (HOG). Alternatively, we will look at learned features by using Principal Components Analysis (PCA). In this case, the algorithm learns the features from analysing the training data.

## 1.1. Handcrafted Features: Histogram of Gradients (HOG)

For the handcrafted features, we opted for the HOG feature descriptor. We tried out both HOG and SIFT, but concluded that HOG yields better results.

First, HOG devides a given picture into connection regions called "cells". For the pixels within each cell, the algorithm compiles a histogram of gradient directions. The descriptor is the concatenation of all these histograms. Ideally, we want our descriptor to be invariant with regards to illumination and shadowing. To achieve this, we can use a process called contrast-normalization. In short, we group multiple cells together in "blocks" for which we calculate a measure of the intensity. This value is then used to normalize all cells within said block. So for an input image of size 64 x 128 x 3, HOG will output a feature vector with a length of 3780.

For more information cfr. https://learnopencv.com/histogram-of-oriented-gradients/.




In [311]:
#step one devide the face into cells
cell_size = 10
if gray_var:
    image_c = cv2.cvtColor(np.uint8(train_X_full[0]), cv2.COLOR_GRAY2RGB)
else:
    image_c =train_X_full[0]
def drawcells(image, dx, dy, grid_color):
    img = image.copy()
    # Modify the image to include the grid
    img[:,::dy,:] = grid_color
    img[::dx,:,:] = grid_color
    return img


cell_ = drawcells(image_c, cell_size,cell_size,[0,255,0])
plt.imshow(cell_)
plt.show()

In [312]:
#create a histogram for the first cell, for simplicity use grayscale

#calculate magnitude and angle of each pixel, ksize=1
def get(X):
    im = X

    # Calculate gradient
    gx = cv2.Sobel(im, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(im, cv2.CV_32F, 0, 1, ksize=1)
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    return mag, angle

if gray_var:
    image = np.uint8(train_X_full[0] )
else:
    image = cv2.cvtColor(np.uint8(train_X_full[0]), cv2.COLOR_RGB2GRAY)

def find_index(array, el):
    for i,l in enumerate(array):
        if l>el:
            break
    return i

mag, angle = get(image)

# the gradient should not differentiate between low->high and high->low
angle = angle % 180
hist_bins = np.array([10,30,50,70,90,110,130,150,170])

cell_angles = angle[0:cell_size,0:cell_size]
cell_magnitudes = mag[0:cell_size,0:cell_size]

hist_values = np.zeros(len(hist_bins))

for row_idx in range(cell_size):
    for col_idx in range(cell_size):
        angle_ = cell_angles[row_idx, col_idx]
        magn_ = cell_magnitudes[row_idx, col_idx]
        i = find_index(hist_bins,angle_)
        hist_values[i]+=magn_

plt.bar(x=hist_bins, height=hist_values, align="center", width=15)
plt.title('Histogram of first cell')
plt.xlabel('Angle Bins')
plt.ylabel('Cum Magnitude')
plt.show()

Ofcourse the values need to be between 0 and 1 so a normalization is required, we could do this by just normalizing this vector or we could take 3 neighbouring vectors as follows and normalize over all 4 of them:

In [313]:
cell__ = drawcells(cell_, cell_size*2,cell_size*2,[255,0,0])
plt.imshow(cell__)
plt.show()

The HOG is calculated on the data shown in the image below, not the picture above.

In [314]:
pixels = 10

fd, hog_image = hog(image_c, orientations=9, pixels_per_cell=(cell_size, cell_size),
                    cells_per_block=(1, 1), visualize=True, channel_axis=-1)
plt.imshow(hog_image, cmap='gray')
plt.show()

All the normalized feature vectors of each red block is then concatenated into one big vector which results in the feature vector.

Let us create the hog features thanks to the skimage library and then visualize then with t-distributed stochastic neighbour embedding (T-SNE). T-SNE is a way to reduce multi-dimensional data. Clustering occers on our train data set, this is a good indication that we can continue with this feature representation.

In [92]:
#T-SNE
from skimage.feature import hog
from sklearn.manifold import TSNE
import seaborn as sns

In [290]:
len(train_y_full)

In [291]:
len(train_X_full)

In [315]:
pixels = 10
hog_df=pd.DataFrame()
hog_df['img']=list(train_X_full)
hog_df['class']=train_y_full
fvs = [None]*len(train_X_full)
for i, img in enumerate(hog_df['img']):
    if gray_var:
        fv = list(hog(img, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                        cells_per_block=(2, 2), feature_vector='True'))
    else:
        fv = list(hog(img, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                        cells_per_block=(2, 2), feature_vector='True', channel_axis =1))
    fvs[i] = fv
hog_df['fv']= fvs

SNE_data = pd.DataFrame()
X_embedded = TSNE(n_components=2, learning_rate=200,
                  init='random', random_state =0).fit_transform(fvs)

#normalization
SNE_data['x']= X_embedded[:,0]+abs(min(X_embedded[:,0]))
SNE_data['X']=SNE_data['x']/max(SNE_data['x'])
SNE_data['y']=X_embedded[:,1]+abs(min(X_embedded[:,1]))
SNE_data['Y']=SNE_data['y']/max(SNE_data['y'])
SNE_data['label']= hog_df['class']

fig, ax = plt.subplots(figsize=(16,10))
sns.scatterplot(x='X', y='Y', data =SNE_data,palette=sns.color_palette("hls", 4), hue='label')

### 1.1.2. HOG: Discussion

**Hyperparameters**

**How to make your descriptor behave well in different circumstances?**

Circumstances can lead to slight perturbations in the look on a person's face, such as:
- Background
-	Face partially covered by objects
- Facial expression
-	General image quality
-	Lighting
- Make-up and hair style
-	Rotation
-	Scale

Some of these perturbations, such as rotation and scale, are taken care of by the preprocessing. Lighting is normalized during the final step of HOG as explained earlier.

**How does this feature compare to your previous grabbing task in the individual assignment?**

The individual assignment requested object grabbing in binary space, with the forground object in white and the background in black. This operation required specific parametrization, depending on the colors of the objects in the frame. Thus, it is only viable in a controlled environment where the luminosity remains constant.

HOG on the other hand can handle complex and multicolor objects rather well. It has proven to be robust both to noise and lighting variations. While both methods can perform object detection, HOG seems to be more reliable.

**Did you need specific pre-processing steps before computing these feature descriptors on your images (which ones and why)?**

Yes, we need to make sure that each picture has the same format. Meaning that the extracted faces need to have the same size and shape, and more important, that all faces are aligned. We do this by giving a specific location for the eyes. Then we rescale the face such that it becomas symmetric.

**Did the visualisation show good discriminative and robustness properties?**


In the T-SNE plot above we can clearly see discriminative clusters Ofcourse this is due the preproccesing of the images. If we would just do hog on raw pictures random background objects would influence the result significant. We saw that when the picture is cropped, small differentiation on the parameters of the location of the eyes or cell size did not change the visualization so starting from basic preprocessed data this technique is quite robust.


## 1.2. Learned Features: Principal Component Analysis (PCA)

PCA is a technique to compress the data from a high dimensional space to a lower one, while keeping most of the variance present in the data. The high-dimensional data is projected on the eigenvectors of the data tensor, which we call the principal components (PCs). The animation below shows the data (blue points) being projected orthogonally on a component (rotating axis). At some point, the component captures maximally the variance in the data. Like that, the data has been compressed from 2 dimensions to 1 dimension. 

![alt text](https://miro.medium.com/max/700/1*XGaA7KWUlhWZLIezYEBIHA.gif)

The PCs are uncorrelated and orthogonal vectors pointing towards where the variance is maximal in the high-dimensional data. The aim is to select the lowest number of PCs such that they are representative well enough the data. These features of the data should be both robust (the features should stay the same for different representations of the same object) and discriminative (the features allow to differentiate different objects).

**PCA**

PCA is based on the Eigenvalue Decomposition (EVD). Let covariance matrix $C$:

$$ C = \frac{X \: X^T}{n-1} $$ 

where $X$ is the data matrix, $C$ is symmetric with shape $p$ x $p$ and hence always diagonalizable. We apply the EVD as follows:

$$ C = W Λ W^T $$

$W$ is an orthogonal matrix and its columns are the eigenvectors of $C$ and $Λ$ is a diagonal matrix containing the corresponding eigenvalues in decreasing order on the diagonal ($λ_1 > λ_2 > λ_3$..). The eigenvectors are the PCs we are seeking. 

**SVD** 

Instead of applying PCA to get the PCs, we performed Singular Values Decomposition (SVD) on the data. SVD is known to be more computationally efficient. 

Any matrix $X$ can be decomposed in the following:

$$ X = U_{n \:x \:n} Σ_{n\: x \:p} V_{p\: x \:n}^T $$

Both $U$ and $V$ are orthogonal and $Σ$ is diagonal. The diagonal elements of Σ are called singular values (σ₁ ≥ σ₂ ≥ … ≥ σₚ ≥ 0).

When comparing SVD to EVD, we observe:

$$ C = \frac{X \: X^T}{n-1} = \frac{V Σ^T U^T U Σ V^T}{n-1} = V\frac{Σ^2}{n-1}V^T $$ 

We can deduce that the columns of $V$ are the PCs and the i-th eigenvalue λᵢ = σᵢ²/(n-1).

We use the svd(X_centered) function of numpy where X is the data matrix which has been centered beforehand. 

**PCA for face recognition**

The PCs capture the features of the faces. We study the influence of the number of PCs used to described the data trhough an analysis of the recontruction error.  


In [23]:
FACE_SIZE = (100, 100)

In [25]:
# Transform all images to an array in 2D (dim: dxd), then transform the array to a single vector of d*d elements 
m_test = len(test_X)

# Dimension of the images
d_test = FACE_SIZE[0]**2

if not gray_var:
    X_gray_test = np.zeros((m_test, *FACE_SIZE))

    # For each, transformation to a 2D array --> grayscale transformation
    for i, img in enumerate(test_X): 
        X_gray_test[i, :] = cv2.cvtColor(np.float32(img), cv2.COLOR_RGB2GRAY).astype(int)

    # Transform matrices to vectors
    X_test_flat = np.reshape(X_gray_test, (m_test, d_test))
else: 
    X_test_flat = np.reshape(test_X, (m_test, d_test))

Let us define an identity feature extractor which returns the input

In [26]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""
    
    def transform(self, X):
        return X
    
    def __call__(self, X):
        return self.transform(X)

Let us define our PCAFeatureExctractor (another possibility, for example, would be to use PCA function from sklearn.decomposition)

In [27]:
class PCAFeatureExtractor(IdentityFeatureExtractor):
    """Extractor of principal components"""
    
    def __init__(self, n_components):
        """initialization of variables"""
        self.n_components = n_components
        self.mean = None
        self.vt = None
        self.principal_components = None
    
    def transform(self, X):
        """Get principal components from images"""
        self.mean = np.mean(X, axis=0, dtype=int) # Or just use that if not conversion to int: X.mean(axis=0)
        X_centered = X - self.mean # Centered image
        
        U, S, Vt = np.linalg.svd(X_centered) # Get SVD from images
        
        self.vt = Vt[:self.n_components, :] # Eigenvectors matrix
        
        self.principal_components = X_centered @ self.vt.T # Get principal components
                
        return self.principal_components
        
    def inverse_transform(self, X):
        """Go back to initial space, back to initial images"""
        raise X @ self.vt - self.mean

Let us calculate the pca components for the train and the test dataset

In [216]:
# Init PCA Feature Extractor + calculate principal components
nbre_features = 10
pca_train = PCAFeatureExtractor(nbre_features)
pca_train.transform(X_flat)

pca_test = PCAFeatureExtractor(nbre_features)
pca_test.transform(X_test_flat)

### 1.2.1. Eigenface Plots

We look at a few eigenfaces. When looking at the first eigenface, we observe a very high contrast between the background and key traits of the human face like the eyes, eyebrows and mouth. This implies that the variance that is captured with this component mostly affects the background. The beauty of PCA is that we immediatly see what **features** the algorithm has learnt.

 We note that later eigenfaces become more and more abstract and **less informative**.

In [ ]:
U, Sigma, VT = np.linalg.svd(X_flat, full_matrices=False)

In [ ]:
plt.set_cmap(plt.cm.gray)
plot_image_sequence_advanced(VT.reshape(VT.shape[0], *(100,100)), VT.shape[0], 10)

Hereafter we plot the **mean eigenface**. 

In [ ]:
mean_face = VT.mean(axis=0)
plt.imshow(mean_face.reshape((100,100)), cmap= plt.cm.gray)

### 1.2.2. Feature Space Plots

In the example above, we performed PCA for all PCs. However, one key idea of PCA is dimensionality reduction. In this section, we study how to keep a minimal number of PCs while keeping most of the variance in the data. 

Hereafter we choose to project the image sample on the 3 first components (the components that explain the most the variance in the data). Therefore, a sample vector of dimension $100^2$ (number of pixel in an image) is reduced to only 3. The plot of the samples in the 3 PCs space is shown below. We observe that there is a separation in the data meaning that the first components seemed to have captured the features of Jesse and Mila.  

In [ ]:
# Create a list of the names

names = list()
for i in train_y_full_trans:
    if i == 1:
        names.append("Jesse")
    elif i == 2:
        names.append("Mila")
    else:
        names.append("Michael and Sarah")

In [ ]:
# Plot the images in a 3D Feature space plot

pca = PCAFeatureExtractor(3)
projected_3D = pca.transform(X_flat)

names = np.asarray(names)


fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection ="3d")

for name in np.unique(names): 
    person = projected_3D[names == name]    
    ax.scatter3D(person[:, 0],
                 person[:, 1],
                 person[:, 2],
                 label= ' '.join(name.split('_'))
                )    

ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_zlabel('PC 3')
ax.legend(loc=1)

plt.show()

We do the same using only the first 2 components. We were able to project high-dimensional data into a 2-dimentional space. This projection shows us that the 2 first components are able to separate a man and a woman. However, our task is to do face recognition, and more PCs may be needed. 

The plot shows us that the data is splitted more along the x-axis, meaning that the **1st component** seems to explain more the variance.

In [ ]:
# Plot the images in a 2D Feature space plot

pca = PCAFeatureExtractor(2)
projected_2D = pca.transform(X_flat)

names = np.asarray(names)

ax = plt.axes()

for name in np.unique(names): 
    person = projected_2D[names == name]  
    plt.scatter(person[:, 0],person[:, 1],label= ' '.join(name.split('_')))
         
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.legend(loc=1)

plt.show()

### Explained variance and number $p$ of *PCs*

The explained variance ratio is computed as:

$$\frac{\Sigma^2}{\Sigma \sigma_{i}}$$

We plot the explained variance ratio in function of the PCs (for the 49 first components). The Figure suggests that the 1st component has a very big weight with respect to the others. 





In [ ]:
explained_variance_ratio = Sigma ** 2 / (Sigma ** 2).sum()
plt.figure(figsize=(16,9))
plt.bar(range(len(Sigma)), explained_variance_ratio)
plt.ylabel('Ratio of variance explained')
plt.xlabel('principal component #')

Hereafyer we plot the cumulative variance explained by the PCs. 

The Figure suggests that 84% of the variance in the data is already explained with the 1st component. With 4 PCs, more than 91% of the variance is explained. 

In [ ]:
cumsum = np.cumsum(explained_variance_ratio)
plt.figure(figsize=(16,9))
plt.plot(range(len(explained_variance_ratio)), cumsum)
plt.ylabel('Cumulative sum of the explained variance')
plt.xlabel('# principal component')

# 2. Evaluation Metrics
## 2.0. Example: Accuracy
As example metric we take the accuracy. Informally, accuracy is the proportion of correct predictions over the total amount of predictions. It is used a lot in classification but it certainly has its disadvantages...

In [37]:
from sklearn.metrics import accuracy_score

# 3. Classifiers
## 3.0. Example: The *'not so smart'* classifier
This random classifier is not very complicated. It makes predictions at random, based on the distribution obseved in the training set. **It thus assumes** that the class labels of the test set will be distributed similarly to the training set.

Let us try a simple example with a random classifier

In [320]:
# Calculate HOG parameters for test data set
from skimage.feature import hog

# pixels = 8
hog_df_test=pd.DataFrame()
hog_df_test['img']=list(train_X_full)
fvs_test = [None]*len(train_X_full)
for i, img in enumerate(hog_df_test['img']):
    if i in face_ind:
        if gray_var:
            fvs_test[i] = list(hog(img, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                    cells_per_block=(2, 2), feature_vector='True'))
        else:
            fvs_test[i] = list(hog(img, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                    cells_per_block=(2, 2), feature_vector='True',channel_axis=channel))
hog_df_test['fv']= fvs_test
hog_train = np.asarray(fvs)

In [321]:
y_pred_hog_test = random_class.predict(hog_df_test['fv'])
for i, cl in enumerate(y_pred_hog_test):
    if cl == 3 or cl == 4:
        y_pred_hog_test[i] = 0
        
print(y_pred_hog_test)

In [322]:
# Calculate accuracy with test dataset
print("accuracy on train dataset: ")
print(accuracy_score(test_y_acc, y_pred_hog_test))

Let us use now a SVM classifier

In [186]:
from sklearn.svm import SVC

In [220]:
# Return a list of featurevectors

def preprocess_(img, size, Gray):
    norm = 100
    gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2GRAY)
    if Gray == False:
        gray = img
        norm = 255
        
    #gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2GRAY)
    fvs = []
    
    #face
    faces = detector(gray, 1)
    for face in faces:
    
        (x, y, w, h) = face.left(), face.top(), face.width(), face.height()
        shape=predictor(gray,face)

        #eyes 
        #right eye, list of all landmarks, then compute center of mass
        re_xvalues = []
        re_yvalues=[]
        for i in range(36,42):
            re_xvalues.append(shape.part(i).x)
            re_yvalues.append(shape.part(i).y)

        le_xvalues = []
        le_yvalues=[]
        for i in range(42,48):
            le_xvalues.append(shape.part(i).x)
            le_yvalues.append(shape.part(i).y)     

        #center of mass
        left_eye = [np.mean(le_xvalues), np.mean(le_yvalues)]
        right_eye = [np.mean(re_xvalues), np.mean(re_yvalues)]

        dX=left_eye[0]-right_eye[0]
        dY = left_eye[1]-right_eye[1]
        angle = np.arctan(dY/dX)

        #calculate translation
        r_eye_loc = 1-l_eye_loc

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = r_eye_loc-l_eye_loc
        desiredDist *= 256
        scale = desiredDist / dist

        eye_center = ((left_eye[0]+right_eye[0])//2 , (left_eye[1]+right_eye[1])//2)

        M = cv2.getRotationMatrix2D(eye_center, math.degrees(angle), scale)

        tX = 256 * 0.5
        tY = 256 * l_eye_loc
        M[0, 2] += (tX - eye_center[0])
        M[1, 2] += (tY - eye_center[1])

        img_rotated = cv2.warpAffine(gray, M, (256, 256)) 


        #normalize
        norm_img = np.zeros((size, size))
        norm_img = cv2.normalize(img_rotated, norm_img, 0, norm, cv2.NORM_MINMAX)

        cropped_im= cv2.resize(norm_img,(size,size),interpolation=cv2.INTER_LINEAR )
#         plt.imshow(cropped_im)
#         plt.show()
        fv = list(hog(cropped_im, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                    cells_per_block=(2, 2), feature_vector='True', channel_axis=channel))
        
        fvs.append(fv) 
        
    return fvs

Let us now do cross validation to find the best SVM parameters

In [162]:
from sklearn.model_selection import cross_val_score

For the determination we use a simple cross-validation technique.

In [190]:
for C_ in [1, 5, 10]:
    for gamma_ in [0.0001, 0.001, 0.01, 0.1]:
        print("C= %0.1f, gamma= %0.4f"% (C_, gamma_))
        clf = SVC(kernel='rbf', C=C_, gamma=gamma_, probability=True)# parameters have been optimized for this training set.
        scores = cross_val_score(clf, hog_train, train_y_full_trans, cv=6)
        print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

The best parameters found are 10 = 5 and gamma = 0.01

In [324]:
# Test SVM classifier
clf = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)# parameters have been optimized for this training set.
clf = clf.fit(hog_train, train_y_full)

To classify the test set each image undergoes a couple of if-statements.
Remember that in the preprocessing part we kept track of the images on which there was not a single face recognized. Based on that we make our first if statement.

if #faces == 1:
    calculate probabilities of each label using our SVM-classifier, if the highest one         among those is higher than the threshold we assign that label to it.
    If it is lower than the threshold we assume that it is a random face so it will be a       zero
if #faces > 1:
    do the above for each face and pick the highest prob among those faces.
    However, if one of the probabilities of label 1 or 2 surpasses the threshold we assign     the corresponding label. This is for images on which there is an 1 and 0.
if #faces == 0:
    assign 0

In [332]:
# Find labels for test dataset
threshold = .85

for i,img in enumerate(test_X):
    if i in face_ind:
        fv = list(hog(img, orientations=9, pixels_per_cell=(pixels,pixels), block_norm='L2',
                    cells_per_block=(2, 2), feature_vector='True',channel_axis=channel))
        
        prob = clf.predict_proba([fv])
        if max(prob[0])>threshold:
            label = np.argmax(prob)
            if label==2 or label==3:
                label = 0
                labels[i] = label
            else:
                label+=1
                labels[i] = label
        else:
            label = 0
            labels[i] = 0
    elif i in faces_ind:
        fv_faces = preprocess_(test['img'][i], size, Gray = gray_var)
        probs = clf.predict_proba(fv_faces)
        highest_prob = threshold
        label_ = 0
        
        # Return face with highest probability
        for p in probs:
            if max(p)>highest_prob:
                highest_prob = max(p)
                label = np.argmax(p)
                if np.argmax(p)==0 or np.argmax(p)==1:
                    break
        if label == 2 or label == 3:
            label=0
            labels[i] = label
        else:
            label +=1
            labels[i] = label
        if i==7:
            
            plt.imshow(test['img'][i])
            plt.show()
            print("Probabilities of Detected Faces:")
            print(probs)
            print("Chosen Label For Picture:")
            print(label)
print(accuracy_score(labels,test_y_acc))

Let us print the result classes

Let us calculate the accuracy

In [333]:
print(accuracy_score(labels,test_y_acc))

In [ ]:
# Print the results

for i, image in enumerate(test_X):
     plt.imshow(image)
     plt.show()
    
     if labels[i]==1:
         print("Jesse")
     elif labels[i]==2:
         print("Mila")
     else:
         print("Michael or Sara")
    

Let us do the same for the PCA features

In [ ]:
clf_pca = SVC(kernel='rbf', C=10, gamma=0.000001, probability=True)# parameters have been optimized for this training set.
scores = cross_val_score(clf_pca, pca_train.principal_components, train_y_full_trans, cv=5)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.37 is best that we can found for PCA with C = 10 and gamma = 0.01

This is due to this data augmentation that we have done which does not work well with PCA (an idea would to replace the bright images by saturate images)

In [ ]:
# Test SVM classifier
clf_pca = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)# parameters have been optimized for this training set.
clf_pca = clf_pca.fit(pca_train.principal_components, train_y_full_trans)

As we notice that HOG works better than PCA, let us quickly calculate the accuracy with the following functions

In [ ]:
y_pred_pca_test = clf_pca.predict(pca_test.principal_components)
# Calculate accuracy with test dataset
print("accuracy on train dataset: ")
print(accuracy_score(y_pred_pca_test,test_y_acc))